<a href="https://colab.research.google.com/github/dhxrth/NUS-SDS-Datathon-Champion/blob/main/NUS_DATATHON_CHAMPION_NUS_146.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Our Analysis

In this analysis, we are analysing sales perfomance of both local and domestic businessess. In this particular case we have combined the domestic and international sales performance of the companies. This motivated by the need for the analysis of the overall sales of a company instead of distinct categories.

To begin the processing, we started with cleaning the data. By removing the data that we thought was unnecccessary, for instance we removed 14,813 rows of data. This data includes, null data under independent factors, latitude, longtitude, accountid, 8digit sic code and its description, company description which made the data set unreliable and unusable, which adversely affects the ability of our modelling.

After removal, we compared the odds ratio of all independent variables to figure out which those are significant enough to be included in our models.

We cannot remove variables removing logistic regression, to identify which variables to remove using p-value as the response variable is not binary.

Odds ratio can only be calculated for ordinal and categorical variables but non of our remaining variables are as such.

By comparing multiple models and comparing their performance indicators such as FPR, FNR, and AUC. We deemed that the best model was KNN as it was the most accurate in predicting the values, which will then help the companies make informed and strategic decisions.

# Data Preparation

The number of empty fields in the "Year Found" category was only around 1.6% of the enire dataset, so those rows were removed. An Age column, which shows the difference between the year the company was found and 2024, as the last column.

The association between the response variables and quantitative input variables were determined using correlation values. The correlation values between the response variable "Domestic Ultimate Sales" and "Age" were tabulated to be -0.0165515 which indicates a weak negative correlation. Since "Global Ultimate Sales" and "Domestic Ultimate Sales" are key response variables, the rows with NA were removed as well. The correlation values between the response variable "Domestic Ultimate Sales" and "Domestic Ultimate Employees" were tabulated to be 0.01638367 which indicates a weak positive correlation. The correlation values between the response variable "Global Ultimate Sales" and "Global Ultimate Employees" were tabulated to be 0.6634766 which indicates a strong positive correlation.

The association between categorical input variables and the response variables were identified using boxplots and the respective means. The mean (means1) Domestic Ultimate Sales was higher by 7.873727 times when the company is not the ultimate or highest-level company within a corporate structure based in its home country (Is Domestic Ultimate = 0). The mean Global Ultimate Sales (means2) was higher by 13.37533 times when Is Global Ultimate = 0. When comparing mean Global Ultimate Sales (means3) and a companies import/export status, companies that do both import and export recorded the highest mean sales. When comparing the industry of a company and their mean Global Ultimate Sales (means4), the top 5 industries were Petroleum Refining, Air Courier Services, Miscellaneous Business Credit Institutions, Plastics Bottles, Television Broadcasting Stations. When comparing the entity type of a company and their mean Global Ultimate Sales (means5), independently owned companies were ranked the highest. When comparing the parent country of a company and their mean Global Ultimate Sales (means6), the top 5 countries were Finland, Mexico, Bahamas, Ireland and Switzerland. When comparing the ownership type of a company and their mean Global Ultimate Sales (means7), privately owned companies were ranked the highest. When comparing the Global Ultimate Country of a company and their mean Global Ultimate Sales (means8), the top 5 countries were Mexico, United States, Japan, Germany and Taiwan.

In [ ]:
set.seed(146)
data = read.csv('catA_train.csv')
getwd()
setwd('/Users/siddharth/Desktop/DSA1101')
dim(data)
head(data)
table(Year.Found)
length(Year.Found)
table(is.na(Year.Found))
434/(434+28748)
#only 1.4% age empty so remove them
data = data[!is.na(data$Year.Found),]
dim(data)
table(is.na(data$Year.Found))
Age = c(2024 - data$Year.Found)
data = cbind(data, Age)
cor(data$Sales..Domestic.Ultimate.Total.USD.,data$Age)
data = data[!is.na(data$Employees..Global.Ultimate.Total.),]
cor(data$Sales..Global.Ultimate.Total.USD.,data$Employees..Global.Ultimate.Total.)
table(is.na(data$Employees..Domestic.Ultimate.Total.))
data = data[!is.na(data$Employees..Domestic.Ultimate.Total.),]
cor(data$Sales..Domestic.Ultimate.Total.USD.,data$Employees..Domestic.Ultimate.Total.)
boxplot(data$Sales..Domestic.Ultimate.Total.USD.~data$Is.Domestic.Ultimate)
means1 <- tapply(data$Sales..Domestic.Ultimate.Total.USD., data$Is.Domestic.Ultimate, mean)
means1
boxplot(data$Sales..Global.Ultimate.Total.USD.~data$Is.Global.Ultimate)
means2 <- tapply(data$Sales..Global.Ultimate.Total.USD., data$Is.Global.Ultimate, mean)
means2
boxplot(data$Sales..Global.Ultimate.Total.USD.~data$Import.Export.Status)
means3 <- tapply(data$Sales..Global.Ultimate.Total.USD., data$Import.Export.Status, mean)
means3
head(table(data$Industry))
boxplot(data$Sales..Global.Ultimate.Total.USD.~data$Industry)
means4 <- tapply(data$Sales..Global.Ultimate.Total.USD., data$Industry, mean)
means4 = sort(means4, decreasing = TRUE)
means4[1:5]
boxplot(data$Sales..Global.Ultimate.Total.USD.~data$Entity.Type)
table(data$Entity.Type)
means5 <- tapply(data$Sales..Global.Ultimate.Total.USD., data$Entity.Type, mean)
means5
boxplot(data$Sales..Global.Ultimate.Total.USD.~data$Parent.Country)
table(data$Parent.Country)
means6 <- tapply(data$Sales..Global.Ultimate.Total.USD., data$Parent.Country, mean)
means6 = sort(means6, decreasing = TRUE)
means6[1:5]
data$Ownership.Type
table(data$Ownership.Type)
boxplot(data$Sales..Global.Ultimate.Total.USD.~data$Ownership.Type)
means7 <- tapply(data$Sales..Global.Ultimate.Total.USD., data$Ownership.Type, mean)
means7
boxplot(data$Sales..Global.Ultimate.Total.USD.~data$Global.Ultimate.Country)
means8 <- tapply(data$Sales..Global.Ultimate.Total.USD., data$Global.Ultimate.Country, mean)
means8 = sort(means8, decreasing = TRUE)
means8[1:5]


# Predictions using Models

In [ ]:
set.seed(1101)
setwd("/Users/tech26/Desktop/NUS/MISC/Datathon")
sales <- read.csv("datathon9.csv")
attach(sales)
# data has been cleaned

sales <- sales[, -which(names(sales) == "SIC.Code")]
# SIC.Code has been removed
sales <- sales[, -which(names(sales) == "X8.Digit.SIC.Description")]
# X8.digit sic description has been removed
sales <- sales[, -which(names(sales) == "X")]
# removing row numbers
sales <- sales[, -which(names(sales) == "Import.Export.Status")]
#removing import export status
sales <- sales[, -which(names(sales) == "Parent.Company")]
# removing Parent.Company
sales <- sales[, -which(names(sales) == "Global.Ultimate.Company")]
#removing Global.Ultimate.Company
sales <- sales[, -which(names(sales) == "Domestic.Ultimate.Company")]
#removing Domestic.Ultimate.Company

dim(sales)
sales$Total_Sales <- sales$Sales..Global.Ultimate.Total.USD. + sales$Sales..Domestic.Ultimate.Total.USD.
# combine the sales data together
sales$Total_Sales <- sales$Sales..Global.Ultimate.Total.USD. + sales$Sales..Domestic.Ultimate.Total.USD.
sales$Total_employess <- sales$Employees..Domestic.Ultimate.Total. + sales$Employees..Global.Ultimate.Total. + sales$Employees..Single.Site.

sales <- sales[, -which(names(sales) == "Sales..Global.Ultimate.Total.USD.")]
sales <- sales[, -which(names(sales) == "Sales..Domestic.Ultimate.Total.USD.")]
# removing the individual sales data

sales <- sales[, -which(names(sales) == "Employees..Domestic.Ultimate.Total.")]
sales <- sales[, -which(names(sales) == "Employees..Global.Ultimate.Total.")]
sales <- sales[, -which(names(sales) == "Employees..Single.Site.")]
# removing the individual employeees data

sales$Entity.Type = as.factor(sales$Entity.Type)
sales$Parent.Country  = as.factor(sales$Parent.Country)
sales$Ownership.Type = as.factor(sales$Ownership.Type)
sales$Global.Ultimate.Country = as.factor(sales$Global.Ultimate.Country)
sales$Is.Domestic.Ultimate = as.factor(sales$Is.Domestic.Ultimate)
sales$Is.Global.Ultimate = as.factor(sales$Is.Global.Ultimate)
sales$Industry = as.factor(sales$Industry)

### ANALYSING DATA SET
dim(sales)
head(sales)
summary(sales)
# min, 1st, median, mean, mean, 3rd, max of each column

# Based on odds ratio of the variables, we reject the variables of odds ratio 0.8-1.2
# Variables  "PhysHlth", "Sex", "Income"as these have a weak association with the reponse variable

### SPLITTING DATA INTO TRAIN AND TEST SETS
# test data : train data <- 1:4
# test and train data will have equal percentage of positive and negative response
n_folds = 5
folds_sales <- sample(rep(1:n_folds, length.out = dim(sales)[1] ))

table(folds_sales)

# negative_data = diabetes[1:35346,] # Only negtaive resposne data


### DECISION TREE CLASSIFIER

acc_1 = numeric(n_folds)
err_1 = numeric(n_folds)
fpr_values_1 = numeric(n_folds)
fnr_values_1 = numeric(n_folds)

library(rpart)
for (j in 1:n_folds){
  test_sales <- which(folds_sales == j)

  train_sales = sales[ -test_sales, ]
  test_sales = sales[test_sales, ]

  train = rbind(train_sales, train_sales)
  test = rbind(test_sales, test_sales)

  model_decision_tree <- rpart(Total_Sales ~ .,
                               method = "class", data =train, control = rpart.control( minsplit =1),
                               parms = list( split ='information'))
  pred = predict(model_decision_tree, newdata = test[,2:14], type = 'class')
  confusion.matrix = table(pred, test[,1])

  TP <- confusion.matrix[2, 2]
  TN <- confusion.matrix[1, 1]
  FP <- confusion.matrix[1, 2]
  FN <- confusion.matrix[2, 1]
  fpr_values_1[j] <- FP / (FP + TN)
  fnr_values_1[j] <- FN / (FN + TP)

  acc_1[j] = sum(diag(confusion.matrix))/sum(confusion.matrix)
  err_1[j] = 1 - sum(diag(confusion.matrix))/sum(confusion.matrix)


}
acc_decision_tree = mean(acc_1); acc_decision_tree # accuracy rate
err_decision_tree = mean(err_1); err_decision_tree # error rate
fpr_values_decision_tree = mean(fpr_values_1); fpr_values_decision_tree # FPR rate
fnr_values_decision_tree = mean(fnr_values_1); fnr_values_decision_tree # FPR rate


library(ROCR)
has_sales <- predict(model_decision_tree, sales[,2:14], type='class')
has_sales = as.numeric(paste(has_sales))
pred_dt = prediction(has_sales, Total_Sales)
roc_dt = performance(pred_dt, measure="tpr", x.measure="fpr")
plot(roc_dt)
auc_dc = performance(pred_dt , measure ="auc")
auc_dc@y.values[[1]] # AUC value

### KNN CLASSIFIER

acc_2 = numeric(n_folds)
err_2 = numeric(n_folds)
fpr_values_2 = numeric(n_folds)
fnr_values_2 = numeric(n_folds)

library(class)
# Doing knn from 1 to 100
K = 100

accuracy = numeric(K)

random <- 5
test_sales <- which(folds_sales == random)

train_sales = negative_data[ -test_sales, ]
test_nej = negative_data[test_sales, ]

train_knn = rbind(train_sales)
test_knn = rbind(test_sales)

# We do the n folds loop outside of the K loop so as to ensrue the program runs in a reasonable amount of time
for (i in 1:K) {
  pred <- knn(train = train_knn[, 2:14], test = test_knn[, 2:14], cl = train_knn[,1], k = i)
  accuracy[i]= mean(test_knn[,1] == pred)
}
which(accuracy == max(accuracy))

# The model that gves the best accuracy is k = 88

for (j in 1:n_folds){
  test_sales <- which(folds_sales == j)

  train_sales = negative_data[ -test_sales, ]
  test_sales = negative_data[test_sales, ]

  train_knn = rbind(train_sales, train_sales)
  test_knn = rbind(test_sales, test_sales)

  model_knn <- knn(train = train_knn[, 2:14], test = test_knn[, 2:14], cl = train_knn$Total_Sales, k = 88)
  confusion.matrix = table(model_knn, test_knn[,1])

  TP <- confusion.matrix[2, 2]
  TN <- confusion.matrix[1, 1]
  FP <- confusion.matrix[1, 2]
  FN <- confusion.matrix[2, 1]
  fpr_values_2[j] <- FP / (FP + TN)
  fnr_values_2[j] <- FN / (FN + TP)

  acc_2[j] = sum(diag(confusion.matrix))/sum(confusion.matrix)
  err_2[j] = 1 - sum(diag(confusion.matrix))/sum(confusion.matrix)
}

acc_knn = mean(acc_2); acc_knn # accuracy rate
err_knn = mean(err_2); err_knn # error rate
fpr_values_knn = mean(fpr_values_2); fpr_values_knn # FPR rate
fnr_values_knn = mean(fnr_values_2); fnr_values_knn # FPR rate

library(ROCR)
library(class)
has_sales <- knn(train = train_knn[, 2:14], test = sales[,2:14], cl = train_knn$Total_Sales, k = 88)
has_sales = as.numeric(has_sales)
pred_knn = prediction(has_sales, Total_Sales)
roc_knn = performance(pred_knn, measure="tpr", x.measure="fpr")
plot(roc_knn)
auc_knn = performance(pred_knn , measure ="auc")
auc_knn@y.values[[1]] # AUC value


### LOGISTIC REGRESSION CLASSIFIER

acc_3 = numeric(n_folds)
err_3 = numeric(n_folds)
fpr_values_3 = numeric(n_folds)
fnr_values_3 = numeric(n_folds)


library(rpart)
library("rpart.plot")
for (j in 1:n_folds){
  test_sales <- which(folds_sales == j)

  train_sales = sales_data[ -test_sales, ]
  test_sales = sales_data[test_sales, ]

  train = rbind(train_sales)
  test = rbind(test_sales)

  model_logistic_regression<- glm(Total_Sales ~ .,
                                  data =train,family = binomial(link ="logit"))
  pred = predict(model_logistic_regression, newdata = test[,2:14], type = 'response')
  pred <- ifelse(pred > 0.5, 1, 0)
  confusion.matrix = table(pred, test[,1])

  TP <- confusion.matrix[2, 2]
  TN <- confusion.matrix[1, 1]
  FP <- confusion.matrix[1, 2]
  FN <- confusion.matrix[2, 1]
  fpr_values_3[j] <- FP / (FP + TN)
  fnr_values_3[j] <- FN / (FN + TP)

  acc_3[j] = sum(diag(confusion.matrix))/sum(confusion.matrix)
  err_3[j] = 1 - sum(diag(confusion.matrix))/sum(confusion.matrix)


}
acc_lr = mean(acc_3); acc_lr # accuracy rate
err_lr = mean(err_3); err_lr# error rate
fpr_values_lr = mean(fpr_values_3); fpr_values_lr # FPR rate
fnr_values_lr = mean(fnr_values_3); fnr_values_lr # FPR rate

library(ROCR)
has_sales = predict(model_logistic_regression, sales[,2:14], type ="response")

pred_lr = prediction(has_sales, Total_Sales)
roc_lr = performance(pred_lr, measure="tpr", x.measure="fpr")
plot(roc_lr)
auc_lr = performance(pred_lr , measure ="auc")
auc_lr@y.values[[1]] # AUC value

### NAIVES BAYERS CLASSIFIER

acc_4 = numeric(n_folds)
err_4 = numeric(n_folds)
fpr_values_4 = numeric(n_folds)
fnr_values_4 = numeric(n_folds)

library(rpart)
for (j in 1:n_folds){
  test_sales <- which(folds_sales == j)

  train_sales = sales_data[ -test_sales, ]
  test_sales = sales_data[test_sales, ]

  train = rbind(train_sales)
  test = rbind(test_sales)

  model_nb<- glm(Total_Sales ~ .,
                 data =train,family = binomial(link ="logit"))
  pred = predict(model_logistic_regression, newdata = test[,2:14], type = 'response')
  pred <- ifelse(pred > 0.5, 1, 0)
  confusion.matrix = table(pred, test[,1])

  TP <- confusion.matrix[2, 2]
  TN <- confusion.matrix[1, 1]
  FP <- confusion.matrix[1, 2]
  FN <- confusion.matrix[2, 1]
  fpr_values_4[j] <- FP / (FP + TN)
  fnr_values_4[j] <- FN / (FN + TP)

  acc_4[j] = sum(diag(confusion.matrix))/sum(confusion.matrix)
  err_4[j] = 1 - sum(diag(confusion.matrix))/sum(confusion.matrix)


}
acc_nb = mean(acc_4); acc_nb # accuracy rate
err_nb = mean(err_4); err_nb # error rate
fpr_values_nb = mean(fpr_values_4); fpr_values_nb # FPR rate
fnr_values_nb = mean(fnr_values_4); fnr_values_nb # FPR rate

library(ROCR)
has_sales = predict(model_nb, sales[,2:14], type ="response")

pred_nb = prediction(has_sales, Total_Sales)
roc_nb = performance(pred_nb, measure="tpr", x.measure="fpr")
plot(roc_nb)
auc_nb = performance(pred_dt , measure ="auc")
auc_nb@y.values[[1]] # AUC value

# COMPARING THE MODELS
# create vector containing all factors to examin goodness of fit
ACCURACY <- c(acc_decision_tree, acc_knn, acc_lr, acc_nb);ACCURACY
ERROR <- c(err_decision_tree, err_knn, err_lr, err_nb);ERROR
FPR <- c(fpr_values_decision_tree, fpr_values_knn, fpr_values_lr, fpr_values_nb);FPR
FNR <- c(fnr_values_decision_tree, fnr_values_knn, fnr_values_lr, fnr_values_nb);FNR
AUC <- c(auc_dc@y.values[[1]], auc_knn@y.values[[1]], auc_lr@y.values[[1]], auc_nb@y.values[[1]]);AUC

highest_accuracy <- which.max(ACCURACY); highest_accuracy
lowest_error <- which.min(ERROR); lowest_error
lowest_fpr <- which.min(FPR); lowest_fpr
lowest_fnr <- which.min(FNR); lowest_fnr
highest_auc <- which.max(AUC); highest_auc

# Therefore it can be concluded that the logistic regression model is the best model

# Comparing predicted response of logistic regression model and thebactual response variable
response_best_model <- predict(model_logistic_regression, newdata = sales, type = 'response')

# Create side-by-side boxplots
boxplot(Total_Sales, response_best_model,
        col = c("blue", "red"),
        xlab = "Response",
        main = "Actual vs Predicted Response")


